In [2]:
from db import db

import pandas as pd
import constants

from bs4 import BeautifulSoup
import requests
import json

from datetime import datetime
import pytz

In [ ]:
# Insert team rosters into the db
for id in constants.nhl_team_ids:
    url = f"https://statsapi.web.nhl.com/api/v1/teams/{id}/roster"
    team_response = dict(requests.get(url).json())
    roster_list = [player['person']['id'] for player in team_response['roster']]
    db.nhl_teams.insert_one({
        "_id": id,
        "roster": roster_list
    })

In [ ]:
# Insert team data into the database
for id in constants.nhl_team_ids:
    url = f"https://statsapi.web.nhl.com/api/v1/teams/{id}"
    team_response = dict(requests.get(url).json())
    db.nhl_teams.update_one({'_id': id}, {'$set': {
        'name': team_response['teams'][0]['name'],
        'abbr': team_response['teams'][0]['abbreviation'],
        'short': team_response['teams'][0]['teamName'],
        'city': team_response['teams'][0]['locationName']
    }})

In [ ]:
# Insert all non-goalie players into db
for team in db.nhl_teams.find():
    for player_id in team['roster']:
        url_bio = f"https://statsapi.web.nhl.com/api/v1/people/{player_id}"
        bio_response = dict(requests.get(url_bio).json())
        url_stats = f"https://statsapi.web.nhl.com/api/v1/people/{player_id}/stats?stats=statsSingleSeason&season=20222023"
        stats_response = dict(requests.get(url_stats).json())
        try:
            if stats_response['stats'][0]['splits']:
                db.nhl_players.insert_one({
                    "_id": player_id,
                    "name": bio_response['people'][0]['fullName'],
                    "number": bio_response['people'][0]['primaryNumber'],
                    "age": bio_response['people'][0]['currentAge'],
                    "nationality": bio_response['people'][0]['nationality'],
                    "height": bio_response['people'][0]['height'],
                    "weight": bio_response['people'][0]['weight'],
                    "captain": bio_response['people'][0]['captain'],
                    "rookie": bio_response['people'][0]['rookie'],
                    "shoots": bio_response['people'][0]['shootsCatches'],
                    "team": bio_response['people'][0]['currentTeam']['id'], 
                    "pos_name": bio_response['people'][0]['primaryPosition']['name'],
                    "pos_type": bio_response['people'][0]['primaryPosition']['type'],
                    "timeOnIce": stats_response['stats'][0]['splits'][0]['stat']['timeOnIce'],
                    "assists": stats_response['stats'][0]['splits'][0]['stat']['assists'],
                    "goals": stats_response['stats'][0]['splits'][0]['stat']['goals'],
                    "pim": stats_response['stats'][0]['splits'][0]['stat']['pim'],
                    "shots": stats_response['stats'][0]['splits'][0]['stat']['shots'], 
                    "games": stats_response['stats'][0]['splits'][0]['stat']['games'],
                    "hits": stats_response['stats'][0]['splits'][0]['stat']['hits'],
                    "powerPlayGoals": stats_response['stats'][0]['splits'][0]['stat']['powerPlayGoals'],
                    "powerPlayPoints": stats_response['stats'][0]['splits'][0]['stat']['powerPlayPoints'],
                    "powerPlayTimeOnIce": stats_response['stats'][0]['splits'][0]['stat']['powerPlayTimeOnIce'],
                    "evenTimeOnIce": stats_response['stats'][0]['splits'][0]['stat']['evenTimeOnIce'],
                    "penaltyMinutes": stats_response['stats'][0]['splits'][0]['stat']['penaltyMinutes'],
                    "faceOffPct": stats_response['stats'][0]['splits'][0]['stat']['faceOffPct'],
                    "shotPct": stats_response['stats'][0]['splits'][0]['stat']['shotPct'],
                    "gameWinningGoals": stats_response['stats'][0]['splits'][0]['stat']['gameWinningGoals'],
                    "overTimeGoals": stats_response['stats'][0]['splits'][0]['stat']['overTimeGoals'],
                    "shortHandedGoals": stats_response['stats'][0]['splits'][0]['stat']['shortHandedGoals'],
                    "shortHandedPoints": stats_response['stats'][0]['splits'][0]['stat']['shortHandedPoints'],
                    "shortHandedTimeOnIce": stats_response['stats'][0]['splits'][0]['stat']['shortHandedTimeOnIce'],
                    "blocked": stats_response['stats'][0]['splits'][0]['stat']['blocked'],
                    "plusMinus": stats_response['stats'][0]['splits'][0]['stat']['plusMinus'],
                    "points": stats_response['stats'][0]['splits'][0]['stat']['points'],
                    "shifts": stats_response['stats'][0]['splits'][0]['stat']['shifts'],
                    "timeOnIcePerGame": stats_response['stats'][0]['splits'][0]['stat']['timeOnIcePerGame'],
                    "evenTimeOnIcePerGame": stats_response['stats'][0]['splits'][0]['stat']['evenTimeOnIcePerGame'],
                    "shortHandedTimeOnIcePerGame": stats_response['stats'][0]['splits'][0]['stat']['shortHandedTimeOnIcePerGame'],
                    "powerPlayTimeOnIcePerGame": stats_response['stats'][0]['splits'][0]['stat']['powerPlayTimeOnIcePerGame']
                })
            else:
                db.nhl_players.insert_one({
                    "_id": player_id,
                    "name": bio_response['people'][0]['fullName'],
                    "number": bio_response['people'][0]['primaryNumber'],
                    "age": bio_response['people'][0]['currentAge'],
                    "nationality": bio_response['people'][0]['nationality'],
                    "height": bio_response['people'][0]['height'],
                    "weight": bio_response['people'][0]['weight'],
                    "captain": bio_response['people'][0]['captain'],
                    "rookie": bio_response['people'][0]['rookie'],
                    "shoots": bio_response['people'][0]['shootsCatches'],
                    "team": bio_response['people'][0]['currentTeam']['id'], 
                    "pos_name": bio_response['people'][0]['primaryPosition']['name'],
                    "pos_type": bio_response['people'][0]['primaryPosition']['type'],
                })
        except:
            print(f"Error with player {player_id}")
            continue

In [ ]:
# Insert all goalies into the database
for player_id in [8477970, 8478406]:
    url_bio = f"https://statsapi.web.nhl.com/api/v1/people/{player_id}"
    bio_response = dict(requests.get(url_bio).json())
    url_stats = f"https://statsapi.web.nhl.com/api/v1/people/{player_id}/stats?stats=statsSingleSeason&season=20222023"
    stats_response = dict(requests.get(url_stats).json())
    try:
        if stats_response['stats'][0]['splits']:
            db.nhl_goalies.insert_one({
                "_id": player_id,
                "name": bio_response['people'][0]['fullName'],
                "number": bio_response['people'][0]['primaryNumber'],
                "age": bio_response['people'][0]['currentAge'],
                "nationality": bio_response['people'][0]['nationality'],
                "height": bio_response['people'][0]['height'],
                "weight": bio_response['people'][0]['weight'],
                "captain": bio_response['people'][0]['captain'],
                "rookie": bio_response['people'][0]['rookie'],
                "shoots": bio_response['people'][0]['shootsCatches'],
                "team": bio_response['people'][0]['currentTeam']['id'], 
                "pos_name": bio_response['people'][0]['primaryPosition']['name'],
                "pos_type": bio_response['people'][0]['primaryPosition']['type'],
                "timeOnIce": stats_response['stats'][0]['splits'][0]['stat']['timeOnIce'],
                "ot": stats_response['stats'][0]['splits'][0]['stat']['ot'],
                "shutouts": stats_response['stats'][0]['splits'][0]['stat']['shutouts'],
                "ties": stats_response['stats'][0]['splits'][0]['stat']['ties'],
                "wins": stats_response['stats'][0]['splits'][0]['stat']['wins'], 
                "losses": stats_response['stats'][0]['splits'][0]['stat']['losses'],
                "saves": stats_response['stats'][0]['splits'][0]['stat']['saves'],
                "powerPlaySaves": stats_response['stats'][0]['splits'][0]['stat']['powerPlaySaves'],
                "shortHandedSaves": stats_response['stats'][0]['splits'][0]['stat']['shortHandedSaves'],
                "evenSaves": stats_response['stats'][0]['splits'][0]['stat']['evenSaves'],
                "shortHandedShots": stats_response['stats'][0]['splits'][0]['stat']['shortHandedShots'],
                "evenShots": stats_response['stats'][0]['splits'][0]['stat']['evenShots'],
                "powerPlayShots": stats_response['stats'][0]['splits'][0]['stat']['powerPlayShots'],
                "savePercentage": stats_response['stats'][0]['splits'][0]['stat']['savePercentage'],
                "goalAgainstAverage": stats_response['stats'][0]['splits'][0]['stat']['goalAgainstAverage'],
                "games": stats_response['stats'][0]['splits'][0]['stat']['games'],
                "gamesStarted": stats_response['stats'][0]['splits'][0]['stat']['gamesStarted'],
                "shotsAgainst": stats_response['stats'][0]['splits'][0]['stat']['shotsAgainst'],
                "goalsAgainst": stats_response['stats'][0]['splits'][0]['stat']['goalsAgainst'],
                "timeOnIcePerGame": stats_response['stats'][0]['splits'][0]['stat']['timeOnIcePerGame'],
                "powerPlaySavePercentage": stats_response['stats'][0]['splits'][0]['stat']['powerPlaySavePercentage'],
                "shortHandedSavePercentage": stats_response['stats'][0]['splits'][0]['stat']['shortHandedSavePercentage'],
                "evenStrengthSavePercentage": stats_response['stats'][0]['splits'][0]['stat']['evenStrengthSavePercentage'],
            })
        else:
            db.nhl_goalies.insert_one({
                "_id": player_id,
                "name": bio_response['people'][0]['fullName'],
                "number": bio_response['people'][0]['primaryNumber'],
                "age": bio_response['people'][0]['currentAge'],
                "nationality": bio_response['people'][0]['nationality'],
                "height": bio_response['people'][0]['height'],
                "weight": bio_response['people'][0]['weight'],
                "captain": bio_response['people'][0]['captain'],
                "rookie": bio_response['people'][0]['rookie'],
                "shoots": bio_response['people'][0]['shootsCatches'],
                "team": bio_response['people'][0]['currentTeam']['id'], 
                "pos_name": bio_response['people'][0]['primaryPosition']['name'],
                "pos_type": bio_response['people'][0]['primaryPosition']['type'],
            })
    except:
        print(f"Error with player {player_id}")
        continue

In [ ]:
cols = ['assists','goals','pim','shots','games','hits','powerPlayGoals','powerPlayPoints']

# Iterate over all players and create leaderboard table
leaderboard_df = []
for player in db.nhl_players.find():
    stats = [player['_id']]
    for col in cols:
        if col in player:
            stats.append(player[col])
        else:
            stats.append(0)
    leaderboard_df.append(stats)
leaderboard_df = pd.DataFrame(leaderboard_df, columns=['id']+cols)

# Create a document of the top 20 players in each category and insert into courtside.nhl_leaderboard collection
for col in cols:
    top_100 = leaderboard_df.sort_values(by=col, ascending=False).head(100)
    db.nhl_leaderboards.insert_one({
        '_id': f"{col}_tot",
        'player_id': top_100['id'].values.tolist(),
        'value': top_100[col].values.tolist(),
        'per_mode': "Totals",
        'name': col,
    })

In [7]:
# Get 2022-23 NHL schedule
url = "https://statsapi.web.nhl.com/api/v1/schedule?startDate=2022-10-07&endDate=2023-04-20"
schedule_response = dict(requests.get(url).json())
for date in schedule_response['dates']:
    for game in date['games']:
        try:
            home = db.nhl_teams.find_one({'_id': game['teams']['home']['team']['id']})
            away = db.nhl_teams.find_one({'_id': game['teams']['away']['team']['id']})
            date = datetime.strptime(game['gameDate'], "%Y-%m-%dT%H:%M:%SZ")
            time = datetime.strptime(game['gameDate'], "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=pytz.UTC)
            eastern_tz = pytz.timezone('US/Eastern')
            db.nhl_schedule.insert_one({
                '_id': f"{home['abbr']}_{away['abbr']}_{date.strftime('%m%d%Y')}",
                'schedule': {
                    'game_date': date.strftime("%m-%d-%Y"),
                    'game_time': time.astimezone(eastern_tz).strftime("%I:%M%p"),
                    'home_code': home['abbr'],
                    'home_name': home['name'],
                    'away_code': away['abbr'],
                    'away_name': away['name'],
                    'arena': game['venue']['name'],
                },
            })
        except:
            print(f"Error with game {game['gamePk']}")
            continue

Error with game 2022040671
Error with game 2022040672
Error with game 2022040673


In [6]:
team_colors = pd.read_csv('old/nhl_colors.csv')

# Iterate over each team and insert hex color codes into nhl_teams collection
for team in db.nhl_teams.find():
    team_color = team_colors[team_colors['team_id'] == team['_id']].values[0][1]
    db.nhl_teams.find_one_and_update({'_id': team['_id']}, {'$set': {'color': team_color}})